<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/main/MVA2024_ex11notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2024 ex11notebookB

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2024

----
## 演習課題: 手計算で / コンピュータで判別分析
---

<b><font color="#ff0000">
注意:
今回の notebook の中には，コードセルを実行すると問題の解答が表示されるようになっている箇所があります．
</font>
</b>


In [ ]:
# いつものいろいろインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# SciPy のもろもろ
import scipy as sp
from scipy.spatial import distance
from scipy.stats import norm, multivariate_normal

# scikit-learn のもろもろ
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix

# 「解答」を示す際に文字列を復号するのに使う
import base64
# 復号した文字列を Markdown 形式で（数式は LaTeX でフォーマットして）表示
from IPython.display import display, Markdown

---
### 手計算で判別分析


手計算で判別分析をやってみましょう．ただし，実際のデータで平均と分散共分散行列を推定するところからやるのはさすがに大変すぎるので，それらは与えられるものとします．

#### 問題1

電卓・コンピュータは使わず，手計算で答えを求めること．

2次元のデータをクラスAとクラスBの2クラスに分類する問題を，判別分析によって解きたい．
それぞれのクラスのデータに正規分布を当てはめると，クラスAの正規分布の平均 $\pmb{\mu}_A$，分散共分散行列 $\Sigma_A$，クラスBの正規分布の平均 $\pmb{\mu}_B$，分散共分散行列 $\Sigma_B$ はそれぞれ次の値だった．


$$
\pmb{\mu}_A = \begin{pmatrix} 3 \\ 4 \end{pmatrix}\qquad
\Sigma_A = \begin{pmatrix}
2 & 1 \\ 1 & 1
\end{pmatrix}\qquad
\pmb{\mu}_B = \begin{pmatrix} 5 \\ 2 \end{pmatrix}\qquad
\Sigma_B = \begin{pmatrix}
1 & 0 \\ 0 & 2
\end{pmatrix}\qquad
$$

このとき，notebookA の式(12)の形の判別関数を用いるなら，$\pmb{x} = \begin{pmatrix} 4 \\ 3 \end{pmatrix}$ はどちらのクラスに属すと予測されるか．

In [ ]:
# 正規分布 A 平均と分散共分散行列
muA = np.array([3.0, 4.0])
covA = np.array([[2.0, 1.0], [1.0, 1.0]])
# 正規分布 B 平均と分散共分散行列
muB = np.array([5.0, 2.0])
covB = np.array([[1.0, 0.0], [0.0, 2.0]])
# 点 P
P = np.array([4, 3])
# グラフを描く
fig, ax = plt.subplots(figsize=(6, 6))
xmin, xmax = -1, 7
ymin, ymax = -1, 7
xx, yy = np.mgrid[xmin:xmax:0.1, xmin:xmax:0.1]
levels = [0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
X = np.dstack((xx, yy)).reshape((-1, 2))
d2 = np.sum(((X - muA) @ np.linalg.inv(covA)) * (X - muA), axis=1)
zz = np.sqrt(d2).reshape((xx.shape[0], yy.shape[0]))
cs = ax.contour(xx, yy, zz, cmap='Blues_r', levels=levels)
ax.clabel(cs)
d2 = np.sum(((X - muB) @ np.linalg.inv(covB)) * (X - muB), axis=1)
zz = np.sqrt(d2).reshape((xx.shape[0], yy.shape[0]))
cs = ax.contour(xx, yy, zz, cmap='Reds_r', levels=levels)
ax.clabel(cs)
ax.scatter(P[0], P[1], s=100, marker='*', color='green', label=r'$\mathbf{x}$')
ax.scatter(muA[0], muA[1], s=100, marker='+', color='blue', label=r'$\mathbf{\mu}_A$')
ax.scatter(muB[0], muB[1], s=100, marker='+', color='red', label=r'$\mathbf{\mu}_B$')
ax.axhline(0, color='gray')
ax.axvline(0, color='gray')
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
ax.set_aspect('equal')
ax.legend()
plt.show()

# ユークリッド距離とマハラノビス距離
tmp = P - muA
edistA = np.sqrt(tmp @ tmp)
tmp = P - muB
edistB = np.sqrt(tmp @ tmp)
mdistA = distance.mahalanobis(P, muA, np.linalg.inv(covA))
mdistB = distance.mahalanobis(P, muB, np.linalg.inv(covB))
print(f'muA との間のユークリッド距離 = {edistA:.4f}   N(muA, covA) との間のマハラノビス距離 = {mdistA:.4f}')
print(f'muB との間のユークリッド距離 = {edistB:.4f}   N(muB, covB) との間のマハラノビス距離 = {mdistB:.4f}')

Q = b'CuWIpOWIpemWouaVsOOCkgoKJCQKXGJlZ2lue2FsaWduZWR9CmgoXHBtYnt4fSkgJj0gLVxmcmFjezF9ezJ9IChccG1ie3h9LVxwbWJ7XG11fV9BKV57XHRvcH1cU2lnbWFfQV57LTF9KFxwbWJ7eH0tXHBtYntcbXV9X0EpCiArIFxmcmFjezF9ezJ9IChccG1ie3h9LVxwbWJ7XG11fV9CKV57XHRvcH1cU2lnbWFfQl57LTF9KFxwbWJ7eH0tXHBtYntcbXV9X0IpXFwKICY9IFxmcmFjezF9ezJ9IFxsZWZ0KCAoXHBtYnt4fS1ccG1ie1xtdX1fQilee1x0b3B9XFNpZ21hX0Jeey0xfShccG1ie3h9LVxwbWJ7XG11fV9CKQogIC0gKFxwbWJ7eH0tXHBtYntcbXV9X0EpXntcdG9wfVxTaWdtYV9BXnstMX0oXHBtYnt4fS1ccG1ie1xtdX1fQSlccmlnaHQpCiBcZW5ke2FsaWduZWR9CiQkCgrjgajjgYrjgY/vvI4KCiQkClxTaWdtYV9BXnstMX0gPSBcYmVnaW57cG1hdHJpeH0gMSAmIC0xIFxcIC0xICYyIFxlbmR7cG1hdHJpeH0gXHFxdWFkClxTaWdtYV9CXnstMX0gPSBcYmVnaW57cG1hdHJpeH0gMSAmIDAgXFwgMCAmIFxmcmFjezF9ezJ9IFxlbmR7cG1hdHJpeH0KJCQKCuOCiOOCiu+8jAoKJCQKXGJlZ2lue2FsaWduZWR9CihccG1ie3h9LVxwbWJ7XG11fV9BKV57XHRvcH1cU2lnbWFfQV57LTF9KFxwbWJ7eH0tXHBtYntcbXV9X0EpICY9ClxiZWdpbntwbWF0cml4fSA0IC0gMyAmIDMgLSA0IFxlbmR7cG1hdHJpeH0gXGJlZ2lue3BtYXRyaXh9IDEgJiAtMSBcXCAtMSAmMiBcZW5ke3BtYXRyaXh9ClxiZWdpbntwbWF0cml4fSA0IC0gMyBcXCAzIC0gNCBcZW5ke3BtYXRyaXh9ID0gNVxcCihccG1ie3h9LVxwbWJ7XG11fV9CKV57XHRvcH1cU2lnbWFfQl57LTF9KFxwbWJ7eH0tXHBtYntcbXV9X0IpICY9ClxiZWdpbntwbWF0cml4fSA0IC0gMyAmIDMgLSA0IFxlbmR7cG1hdHJpeH0gXGJlZ2lue3BtYXRyaXh9IDEgJiAwIFxcIDAgJiBcZnJhY3sxfXsyfSBcZW5ke3BtYXRyaXh9ClxiZWdpbntwbWF0cml4fSA0IC0gMyBcXCAzIC0gNCBcZW5ke3BtYXRyaXh9ID0gXGZyYWN7M317Mn1cXApcZW5ke2FsaWduZWR9CiQkCgrjgafjgYLjgovvvI7jgZfjgZ/jgYzjgaPjgabvvIwkaChccG1ie3h9KSA8IDAkIOOBqOOBquOCi+OBruOBp++8jCRccG1ie3h9JCDjga/jgq/jg6njgrlC44Gr5bGe44GZ44KL77yOCgrlm7Pjga7nrYnpq5jnt5rjgpLopovjgabjgoLjgYrjgojjgZ3jga7lgKTjga/liIbjgYvjgovjgYzvvIwKJFxwbWJ7eH0kIOOBqOOCr+ODqeOCuUHjga7mraPopo/liIbluIPjgajjga7plpPjga7jg57jg4/jg6njg47jg5Pjgrnot53pm6Ljga8gJFxzcXJ0ezV9IFxhcHByb3ggMi4yNCTvvIwKJFxwbWJ7eH0kIOOBqOOCr+ODqeOCuULjga7mraPopo/liIbluIPjgajjga7plpPjga7jg57jg4/jg6njg47jg5Pjgrnot53pm6Ljga8gJFxzcXJ0e1xmcmFjezN9ezJ9fSBcYXBwcm94IDEuMjIkIOOBp+OBguOCiwoK'
display(Markdown(base64.b64decode(Q).decode('utf-8')))

---
### コンピュータで判別分析

notebookA で行った「人間 vs ほげ星人」の判別分析の実験を，[scikit-learn](https://scikit-learn.org/) という機械学習ライブラリを利用して，もっと簡単にやってみましょう．



まずはデータを用意します．配列 `X` に (身長, 体重) の値を，配列 `y` に Human か Hoge かを表す値（0ならHuman，1ならHoge）を，それぞれ代入しておきます．

In [ ]:
# 人間 vs ほげ星人
URL = 'https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/humanvshoge.csv'
dfHoge = pd.read_csv(URL)
#dfHoge

# (身長, 体重) の2次元データ
X = dfHoge[['Height', 'Weight']].to_numpy()
# クラスラベル  Human が 0 で Hoge が 1
y = (dfHoge['Class'] == 'Hoge').to_numpy().astype(int)

# 最初の5件を表示
print(' X[n, 0] X[n, 1] y[n]')
for n in range(5):
    print(f'{n} {X[n, 0]}   {X[n, 1]}    {y[n]}')
print(f'X.shape = {X.shape}')

次のコードセルで判別分析を実行し，結果を表示します．
ここでは，scikit-learn に含まれる判別分析の機能 [sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis.html) を用いています．
この場合，notebookA の式(8)および(11)を判別関数とすることに相当します．

In [ ]:
# 判別分析のための QuadraticDiscriminantAnalysis クラスのインスタンスを生成
qda = QuadraticDiscriminantAnalysis(priors=(0.5, 0.5))

# X, y を用いて正規分布のパラメータを推定
qda.fit(X, y)

# X に含まれる個々のデータの所属クラスを予測
y_pred = qda.predict(X)

# 結果を表示
print(' X[n, 0] X[n, 1] y[n]  y_pred[n]')
for n in range(5):
    print(f'{n} {X[n, 0]}   {X[n, 1]}    {y[n]}   {y_pred[n]}')

# 混同行列を求めて Markdown の表形式で表示
confusion = confusion_matrix(y, y_pred)
ss = f'''
| |予測が Human|予測が Hoge|
|:--|--:|--:|
|**正解が Human**|{confusion[0, 0]}|{confusion[0, 1]}|
|**正解が Hoge**|{confusion[1, 0]}|{confusion[1, 1]}|
'''
display(Markdown(ss))

全てのデータのクラスを正しく予測できていますね．

次は，所属するクラスが未知のデータを用意して，それらのクラスを予測させてみましょう．

In [ ]:
# 新しいデータ
X2 = np.array([[150.0,  40.0],
               [150.0,  60.0],
               [150.0,  80.0],
               [150.0, 100.0],
               [150.0, 120.0]])

次のコードセルで予測を実行し，結果を表示します．
このコードセルでは，正規分布のパラメータ（平均と分散共分散行列）を推定する計算は行わないことに注意してください．その計算は，所属クラスの正解を知っているデータ `X` と `y` を使ってすでに実行済みです．ここでは，そこで求めたパラメータを用いてクラスの予測を行っています．つまり，判別関数の計算と，その符号に基づいて所属クラスを決める計算です．

In [ ]:
# X2 に含まれる個々のデータの所属クラスを予測
y2_pred = qda.predict(X2)
# 結果を表示
print(' X2[n, 0] X2[n, 1]  y2_pred[n]')
for n in range(X2.shape[0]):
    print(f'{n} {X2[n, 0]}   {X2[n, 1]}    {y2_pred[n]}')